In [1]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import tensorflow as tf
import netCDF4
from tensorflow.keras.models import load_model

2024-10-01 18:23:42.449139: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 18:23:42.462561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 18:23:42.478211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 18:23:42.482847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 18:23:42.494060: I tensorflow/core/platform/cpu_feature_guar

In [2]:
@tf.function()
def compute_divflux(u, v, h, dx, dy):
    """
    Upwind computation of the divergence of the flux: d(u h)/dx + d(v h)/dy
    
    Parameters:
    - u: x-component of velocity (2D tensor).
    - v: y-component of velocity (2D tensor).
    - h: ice thickness (2D tensor).
    - dx: grid spacing in the x-direction (float).
    - dy: grid spacing in the y-direction (float).
    
    Returns:
    - divflux: divergence of flux (2D tensor).
    """
    # Compute u and v on the staggered grid
    u = tf.concat([u[:, 0:1], 0.5 * (u[:, :-1] + u[:, 1:]), u[:, -1:]], 1)  # shape (ny, nx+1)
    v = tf.concat([v[0:1, :], 0.5 * (v[:-1, :] + v[1:, :]), v[-1:, :]], 0)  # shape (ny+1, nx)

    # Extend h with constant value at the domain boundaries
    Hx = tf.pad(h, [[0, 0], [1, 1]], "CONSTANT")  # shape (ny, nx+2)
    Hy = tf.pad(h, [[1, 1], [0, 0]], "CONSTANT")  # shape (ny+2, nx)

    # Compute fluxes by selecting the upwind quantities
    Qx = u * tf.where(u > 0, Hx[:, :-1], Hx[:, 1:])  # shape (ny, nx+1)
    Qy = v * tf.where(v > 0, Hy[:-1, :], Hy[1:, :])  # shape (ny+1, nx)

    # Compute the divergence, final shape is (ny, nx)
    divflux = (Qx[:, 1:] - Qx[:, :-1]) / dx + (Qy[1:, :] - Qy[:-1, :]) / dy
    return divflux


@tf.function()
def compute_gradient_tf(s, dx, dy):
    """
    Compute spatial 2D gradient of a given field.
    
    Parameters:
    - s: surface elevation (2D tensor).
    - dx: grid spacing in the x-direction (float).
    - dy: grid spacing in the y-direction (float).
    
    Returns:
    - diffx: gradient in the x-direction (2D tensor).
    - diffy: gradient in the y-direction (2D tensor).
    """
    EX = tf.concat([1.5 * s[:, 0:1] - 0.5 * s[:, 1:2], 0.5 * s[:, :-1] + 0.5 * s[:, 1:], 1.5 * s[:, -1:] - 0.5 * s[:, -2:-1]], 1)
    diffx = (EX[:, 1:] - EX[:, :-1]) / dx

    EY = tf.concat([1.5 * s[0:1, :] - 0.5 * s[1:2, :], 0.5 * s[:-1, :] + 0.5 * s[1:, :], 1.5 * s[-1:, :] - 0.5 * s[-2:-1, :]], 0)
    diffy = (EY[1:, :] - EY[:-1, :]) / dy

    return diffx, diffy


def apply_boundary_condition(H_ice, boundary_width=5):
    """
    Apply boundary condition to the ice thickness field `H_ice`.
    The ice thickness will linearly decrease to zero starting from `boundary_width` pixels away from the boundary.
    
    Parameters:
    - H_ice: 2D numpy array representing ice thickness.
    - boundary_width: Number of pixels from the boundary where H_ice starts to decrease.
    
    Returns:
    - Modified H_ice with boundary condition applied.
    """
    ny, nx = H_ice.shape  # Get the dimensions of the ice thickness field

    # Create linear ramps
    ramp = np.linspace(1, 0, boundary_width)  # Ramp that linearly decreases from 1 to 0

    # Apply boundary condition to the left boundary
    H_ice[:, :boundary_width] *= ramp[::-1]  # Decrease from boundary to 5 pixels inwards

    # Apply boundary condition to the right boundary
    H_ice[:, -boundary_width:] *= ramp  # Decrease from 5 pixels inwards to the boundary

    # Apply boundary condition to the top boundary
    H_ice[:boundary_width, :] *= ramp[::-1, np.newaxis]  # Decrease vertically from top boundary

    # Apply boundary condition to the bottom boundary
    H_ice[-boundary_width:, :] *= ramp[:, np.newaxis]  # Decrease vertically to bottom boundary
    
    return H_ice


In [3]:


def visualize_glacier(H_ice, Z_surf, time, Lx, Ly):
    """
    Visualize glacier surface and ice thickness.
    
    Parameters:
    - H_ice: Ice thickness (2D numpy array).
    - Z_surf: Ice surface elevation (2D numpy array).
    - time: Current simulation time (float).
    - Lx: Glacier length in x-direction (float).
    - Ly: Glacier length in y-direction (float).
    """
    clear_output(wait=True)  # Clear the previous output in the notebook
    
    plt.figure(2, figsize=(11, 4), dpi=200)
    
    # First subplot: Ice surface
    plt.subplot(1, 2, 1)
    plt.imshow(Z_surf, extent=[0, Lx/1000, 0, Ly/1000], cmap='terrain', origin='lower')
    plt.colorbar(label='Elevation (m)')
    plt.title('Ice Surface at ' + str(int(time)) + ' y')
    plt.xlabel('Distance, km')
    plt.ylabel('Distance, km')

    # Second subplot: Ice thickness
    plt.subplot(1, 2, 2)
    plt.imshow(np.where(H_ice > 0, H_ice, np.nan), extent=[0, Lx/1000, 0, Ly/1000], cmap='jet', origin='lower')
    plt.colorbar(label='Ice Thickness (m)')
    plt.title('Ice Thickness at ' + str(int(time)) + ' y')
    plt.xlabel('Distance, km')
    plt.ylabel('Distance, km')

    # Show the plot
    plt.show()

In [4]:
def forward_simulation_tf(b_max, grad_b, Z_ELA, model, scaling_factors, H_ice, Z_topo, 
                          ttot, dtmax, cfl, dx, dy, nx, ny):
    """
    Perform forward simulation of glacier thickness evolution in 2D using the emulator for velocity.
    
    Parameters:
    - b_max: Maximum mass balance (float).
    - grad_b: Gradient of mass balance (float).
    - Z_ELA: Elevation of equilibrium line altitude (trainable tf.Variable).
    - model: Pre-trained neural network model for velocity prediction.
    - scaling_factors: Dictionary with scaling factors for inputs and outputs of the model.
    - H_ice: Initial ice thickness (2D tf.Tensor).
    - Z_topo: Topography (2D tf.Tensor).
    - ttot: Total simulation time (float).
    - dtmax: Maximum time step (float).
    - cfl: CFL condition constant (float).
    - dx, dy: Grid spacing in x and y directions (float).
    - nx, ny: Number of grid points in x and y directions (int).
    
    Returns:
    - H_ice: Final ice thickness after simulation (2D tf.Tensor).
    """
    # Initial time and iteration count
    time = tf.cast(0.0, tf.float32)
    dt = tf.cast(dtmax, tf.float32)
    it = 0

    # Ensure the input fields are in float32
    H_ice = tf.cast(H_ice, tf.float32)
    Z_topo = tf.cast(Z_topo, tf.float32)
    
    # Start the simulation loop
    while time < ttot:
        # Step 1: Compute surface elevation and gradients (slopes)
        Z_surf = Z_topo + H_ice
        slopsurfx, slopsurfy = compute_gradient_tf(Z_surf, dx, dy)
        
        # Step 2: Scale inputs for the emulator (ML model)
        H_ice_scaled = H_ice / scaling_factors["thk"]
        slopsurfx_scaled = slopsurfx / scaling_factors["slopsurfx"]
        slopsurfy_scaled = slopsurfy / scaling_factors["slopsurfy"]
        
        # Combine inputs and predict velocity
        input_data_scaled = tf.stack([H_ice_scaled, slopsurfx_scaled, slopsurfy_scaled], axis=-1)
        input_data_scaled = tf.expand_dims(input_data_scaled, axis=0)  # Add batch dimension
        ubar_vbar_pred = model(input_data_scaled, training=False)
        ubar = ubar_vbar_pred[0, :, :, 0] * scaling_factors["ubar"]
        vbar = ubar_vbar_pred[0, :, :, 1] * scaling_factors["vbar"]

        # Step 3: Compute maximum velocity for CFL condition
        vel_max = tf.reduce_max([tf.reduce_max(tf.abs(ubar)), tf.reduce_max(tf.abs(vbar))])
        dt = tf.cast(tf.minimum(cfl * dx / vel_max, dtmax), tf.float32)

        # Step 4: Compute the change in thickness (dH/dt)
        dHdt = -compute_divflux(ubar, vbar, H_ice, dx, dy)
        
        # Update ice thickness and ensure it's non-negative
        H_ice = H_ice + dt * dHdt
        H_ice = tf.maximum(H_ice, 0.0)

        # Step 5: Apply Surface Mass Balance (SMB) rule
        b = tf.minimum(grad_b * (Z_surf - Z_ELA), b_max)
        H_ice += dt * b

        # Apply boundary condition to ensure no ice at domain boundaries
        H_ice = apply_boundary_condition(H_ice)

        # Update time and iteration count
        time += dt
        it += 1

    return H_ice

In [8]:
# Physical parameters
Lx = 49700    # Domain length in x (m)
Ly = 32300    # Domain length in y (m)
ttot = 700   # Time limit (yr)
grad_b = 0.001 # Mass balance gradient (no unit)
b_max = 0.5   # Maximum precip (m/yr)
Z_ELA = 3000  # Elevation of equilibrium line altitude (m)
rho = 910.0   # Ice density (g/m^3)
g   = 9.81    # Earth's gravity (m/s^2)
fd  = 1e-18   # Deformation constant (Pa^-3 y^-1) 



# Initialization & load data

nout = 50  # Frequency of plotting
dtmax = 1   # maximum time step
dt = dtmax  # Initial time step
dx = 100
dy = 100  # Cell size in y
cfl = 0.20
nx=int(Lx/dx)
ny=int(Ly/dy)
x = np.linspace(0, Lx, nx)  # x-coordinates
y = np.linspace(0, Ly, ny)  # y-coordinates

nc_file = netCDF4.Dataset('bedrock.nc')  # Load the NetCDF file
Z_topo = nc_file.variables['topg']    # Replace 'topg' with the appropriate v
model = load_model('glacier_flow_model.h5')

H_ice = np.zeros((ny, nx))  # Initial ice thickness
Z_surf = Z_topo + H_ice  # Initial ice surface
time = 0  # Initial time
it = 0

scaling_factors= {'thk': 780.0,
 'slopsurfx': 1.8200000524520874,
 'slopsurfy': 1.6096999645233154,
 'ubar': 346.2121276855469,
 'vbar': 173.1743927001953}

2024-10-01 18:28:02.572802: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleLoadData(&module, data)' failed with 'CUDA_ERROR_UNSUPPORTED_PTX_VERSION'

2024-10-01 18:28:02.572829: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'

2024-10-01 18:28:02.572839: W tensorflow/core/framework/op_kernel.cc:1828] INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE'
2024-10-01 18:28:02.572853: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INTERNAL: 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE'


InternalError: {{function_node __wrapped__FloorMod_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:FloorMod] name: 

In [15]:
forward_simulation_tf(b_max, grad_b, Z_ELA, model, scaling_factors, H_ice, Z_topo, 
                          ttot, dtmax, cfl, dx, dy, nx, ny)

TypeError: forward_simulation_tf() missing 14 required positional arguments: 'b_max', 'grad_b', 'Z_ELA', 'model', 'scaling_factors', 'H_ice', 'Z_topo', 'ttot', 'dtmax', 'cfl', 'dx', 'dy', 'nx', and 'ny'